In [ ]:
from scipy.sparse import csr_matrix, lil_matrix, load_npz, save_npz, hstack
import numpy as np 
import pandas as pd 
from tqdm import tqdm 

In [ ]:
path = '/projects/melloddy/data/y2run_final/data_prep/y2_ext/y2_ext_noaux/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [ ]:
path = './aux_data/baseline_plus_aux_data/matrices/cls/cls_T10_y.npz'
y_aux = load_npz(path)

In [ ]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

In [ ]:
# Generate the y-sparse matrix for inference 
# Main tasks + fold 0 to limit the amount of predictions

In [ ]:
path = './aux_data/baseline_plus_aux_data/matrices/cls/cls_T11_fold_vector.npy'
folds = np.load(path)

In [ ]:
path = '../notebooks/aux_data/baseline_plus_aux_data/results_tmp/classification/T8c.csv'
t8c = pd.read_csv(path)

In [ ]:
path = './cp/summary_eps_0.05_mgd_cp.csv'
df_cp_mgd = pd.read_csv(path)

In [ ]:
df_matching = pd.merge(
    df_cp_mgd
    ,t8c
    ,on=['input_assay_id','threshold']
    ,how='inner'
    ,suffixes=('_image','_baselineaux')
)

In [ ]:
# main tasks 
# mind the difference between these two !
# cont_classification_task_id is the baseline + aux model 
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id'].tail())
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id_baseline'].tail())

In [ ]:
df_matching.shape

In [ ]:
path = './mapping/baseline_image_model_baselineaux_task_mapping.csv'
df_matching.to_csv(path, index=False)

In [ ]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [ ]:
for i,l1 in tqdm(enumerate(lst)):
    l2 = l1
    print(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'].nunique())

In [ ]:
# new approach : for inference, only consider the main tasks that correspond to aux tasks meeting the criteria 
# instead of one general sparse y matrix 



for i,l1 in tqdm(enumerate(lst)):
    y_sparse_main_fold0 = lil_matrix(y_aux.shape)
    print(l1)
    l2 = l1
    #for j,l2 in enumerate(lst): 
    file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
    col_idxs = set(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'])
    for col in col_idxs :
        y_sparse_main_fold0[folds == 0,col] = 1
    path = './aux_data/baseline_plus_aux_data/matrices/cls/y_sparse_main_tasks_fold0_{}'.format(file_core)
    save_npz(path,csr_matrix(y_sparse_main_fold0))